In [1]:
### Can probably cut these down as we are no longer
### running the training in this notebook

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn 
import librosa
import librosa.display as display
import os
import warnings
import matplotlib
from tqdm import tqdm
import multiprocessing as mp
from shutil import copyfile
import math


from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise
from kapre.time_frequency import Spectrogram

%matplotlib inline

#!rm -r train_data
#!rm -r val_data
#!rm -r models
#!mkdir models

# suppress warnings
warnings.filterwarnings("ignore")

# NOT USING THE GPU
# SOUND_DIR = "/project/data/birdsong-recognition/train_audio/"
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

Using TensorFlow backend.


In [2]:
# function for creating Mel Spectrogram
def createMelSpectrogram(input_path, fileName, output_path, saveOrShow=0, output_path_remote = None, resolution = None):
    
    input_file=os.path.join(output_path, fileName)
    output_file=os.path.join(output_path, fileName[:-4] + ".png")
    print(output_file, ) 

    if ( not( os.path.exists(output_file)) and not(os.path.isdir(output_file))):
        print(f"{output_file} does not exist, and is not a directory")
        # load sound signal
        signal, sr = librosa.load(os.path.join(input_path, fileName), duration=10)

        # print(signal.shape)
        # print(signal.shape[0] / resolution )
        
        # Calculate the n_hop to get the specified resolution
        if (resolution is None):
            num_hops = 256
            resolution = 224
        else:
            num_hops = int(math.ceil(signal.shape[0] / resolution )) # ceil ensures exact resolution
        
        # create Mel Spectrogram
        # print(f"Creating spectrogram {output_file}, num_hops: {num_hops}, calcualted samples = {num_hops*resolution}, actual samples = {signal.shape[0]}")
        S = Melspectrogram(n_dft=1024, 
                           n_hop=num_hops,
                           input_shape=(1, signal.shape[0]),
                           padding='same', sr=sr, n_mels=resolution, fmin=1400, fmax=sr/2,
                           power_melgram=2.0, return_decibel_melgram=True,
                           trainable_fb=False, trainable_kernel=False)(signal.reshape(1, 1, -1)).numpy()

        print ("Before:",S.shape)
        
        S = S.reshape(S.shape[1], S.shape[2])
        
        print ("After:",S.shape, S.shape[1], resolution)
        
        # because the steps dont always work out exactly, pad with zeros to complete the shape
        if (S.shape[1] < resolution):
            S = np.hstack( (S, np.zeros( (S.shape[0], resolution - S.shape[1])) ) ) 
            print("padded to:", S.shape, "for", resolution)
        
        if saveOrShow == 0:   
            matplotlib.image.imsave(output_file, S)
            if (output_path_remote is not None):
                output_file_remote=os.path.join(output_path_remote, fileName[:-4] + ".png")
                print("Remote Output file:", output_file_remote)
                matplotlib.image.imsave(output_file_remote, S)
                
        else:
            #plt.imshow(S)
            #plt.show()
            display.specshow(S, sr=sr)
            plt.show()

## Create Mel-Spectrograms and split for training and validation

In [3]:
# select top 20 birds for training
# BIRDS = os.listdir("/project/data/birdsong-recognition/train_audio/")[0:20]
BIRDS = ["amered",   "blugrb1",  "commer",  "horlar",  "pasfly",  "vigswa",  "wilsni1", \
         "annhum",   "brthum",   "gockin",  "moudov",  "semsan",  "wewpew",  "yelwar",\
         "belkin1",  "cedwax",   "gryfly",  "olsfly", "sposan",  "whbnut" ]


# BIRDS = ['amered']

In [4]:
print(BIRDS)

['amered', 'blugrb1', 'commer', 'horlar', 'pasfly', 'vigswa', 'wilsni1', 'annhum', 'brthum', 'gockin', 'moudov', 'semsan', 'wewpew', 'yelwar', 'belkin1', 'cedwax', 'gryfly', 'olsfly', 'sposan', 'whbnut']


In [5]:
# set the size of spectragrams here. The spectrograms will be
# square, and with 3 colour channels (RGB): (spsize,spsize,3)
# spsize = 224
# spsize = 299
spsize = 331


train_folder = "/project/data/train_data_decibel_20/"
val_folder = "/project/data/val_data_decibel_20/"

train_folder_remote = "/data/train_data_decibel_20/"
val_folder_remote = "/data/val_data_decibel_20/"


if not os.path.exists(train_folder + str(spsize)): os.mkdir(train_folder + str(spsize))
if not os.path.exists(val_folder + str(spsize)): os.mkdir(val_folder + str(spsize))

In [6]:
%%time


# create train and val spectrogram
np.random.seed(1234)
def process_bird(bird):
    INPUT_DIR = os.path.join("/project/data/birdsong-recognition/train_audio/", bird)
    TRAIN_DIR = os.path.join(train_folder, str(spsize), bird)
    VAL_DIR = os.path.join(val_folder, str(spsize), bird)
    
    # TRAIN_DIR_REMOTE = os.path.join(train_folder_remote, str(spsize), bird)
    # VAL_DIR_REMOTE = os.path.join(val_folder_remote, str(spsize), bird)
    
    
    TRAIN_DIR_OLD = os.path.join(train_folder, bird)
    VAL_DIR_OLD = os.path.join(val_folder, bird)
    
    # create bird folders if they don't already exist
    if not(os.path.exists(TRAIN_DIR)):
        os.mkdir(TRAIN_DIR)
    
    if not(os.path.exists(VAL_DIR)): 
        os.mkdir(VAL_DIR)
        
    # if not(os.path.exists(TRAIN_DIR_REMOTE)):
    #    os.mkdir(TRAIN_DIR_REMOTE)
    
    # if not(os.path.exists(VAL_DIR_REMOTE)): 
    #    os.mkdir(VAL_DIR_REMOTE)
        
    # to avoid valdiation and training sets having similar recordings, we will
    # separate them based on the core part of the filename: XCnnnnnn. 
    # We keep track of the cores already sent to each of train and validation sets
    
    # First, build the list of core names currently in the validation and training dirs
    # by stripping off the ending.
    
    trn_names = []
    val_names = []
    
   
    for file in [ f for f in os.listdir(TRAIN_DIR_OLD) if f[0] == "X" ]:
        trn_names.append(file.split(".")[0])
    
    for file in [ f for f in os.listdir(VAL_DIR_OLD) if f[0] == "X" ]:
        val_names.append(file.split(".")[0])
    
    # print("trn_names", trn_names)
    # print("val_names", val_names)
    
    # cross check that nothing appears in both lists
    for core_name in val_names:
        if core_name in trn_names:
            print(f"core name {core_name} appears in both training and validation sets")

    # process file-by-file
            
    # look at each of the the original files (starting with "X")
    # if they are already assigned, the spectragram of the file will
    # be sent to the same data set.
    
        
    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "X"]:
        core_name = f.split(".")[0]
        # this is an original file. Check if it has already been assigned
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
#            dest_dir_remote = TRAIN_DIR_REMOTE
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            print(f"{core_name} already in validation dataset")
            dest_dir = VAL_DIR
#            dest_dir_remote = VAL_DIR_REMOTE
        else:
            # this is an original file we haven't assigned to a dataset
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"Randomly assigning {core_name} to training dataset")
            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"Randomly assigning {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir, resolution = spsize )
    
    # Now do the same for the generated files

    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "A"]:
        core_name = f.split(".")[1] # discards the leading "A."
        
        # check if it has already been assigned.
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
#            dest_dir_remote = TRAIN_DIR_REMOTE
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            dest_dir = VAL_DIR
#            dest_dir_remote = VAL_DIR_REMOTE
            print(f"{core_name} already in validation dataset")
        else:
            # THIS SHOULD NOT HAPPEN!!!
            # if we get here, then the original XCnnnnnn.png has
            # gone missing, so print a warning and assign randomly
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to training dataset")

            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir, resolution = spsize )
            
            

CPU times: user 48 µs, sys: 0 ns, total: 48 µs
Wall time: 52.5 µs


In [7]:
%%time

def main():

    np.random.seed(1234)
    # comment out the next line and uncomment the following one if running
    # on non-hyperthreaded CPU cores. Assigns one thread per physical core
    # reserving one core for OS, IO, compression etc. Cloud does not have
    # hyperthreading, so adjusted for that.
    
    # threads = int((mp.cpu_count() /2) - 1)
    threads = int(mp.cpu_count() -1)
    
    print(f"Using {threads} threads.")
    # Handle single-core machines
    if (threads < 1): threads = 1

    pool = mp.Pool(threads)
    
    pool.map(process_bird, BIRDS)


        
       

if __name__ == "__main__":
    main()
                   
        

Using 7 threads.
XC109942 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC109942.png
XC109943 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC109943.png
XC109945 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC109945.png
XC128945 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC128945.png
XC139865 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC139865.png
XC139884 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC139884.png
XC140266 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC140266.0.png
XC123985 already in training dataset/project/data/train_data_decibel_20/331/vigswa/XC140266.0.png does not exist, and is not a directory

/project/data/train_data_decibel_20/331/pasfly/XC123985.png
XC123997 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC123997.png
XC12399

/project/data/train_data_decibel_20/331/pasfly/XC164567.png
XC164569 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC164569.png
XC169027 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC169027.png
XC174581 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC174581.png
XC178530 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC178530.png
XC178942 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC178942.0.png
/project/data/train_data_decibel_20/331/pasfly/XC178942.0.png does not exist, and is not a directory
XC112579 already in training dataset
/project/data/train_data_decibel_20/331/blugrb1/XC112579.png
XC123302 already in training dataset
/project/data/train_data_decibel_20/331/blugrb1/XC123302.png
XC124268 already in training dataset
/project/data/train_data_decibel_20/331/blugrb1/XC124268.0.png
XC164307 already in training dataset
/project/data/train

/project/data/train_data_decibel_20/331/amered/XC134499.1.png
/project/data/train_data_decibel_20/331/amered/XC134499.1.png does not exist, and is not a directory
XC243693 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC243693.1.png
XC134882 already in training dataset
/project/data/train_data_decibel_20/331/commer/XC134882.1.png
XC234332 already in training dataset
/project/data/train_data_decibel_20/331/pasfly/XC234332.1.png
/project/data/train_data_decibel_20/331/vigswa/XC243693.1.png does not exist, and is not a directory
/project/data/train_data_decibel_20/331/commer/XC134882.1.png does not exist, and is not a directory
/project/data/train_data_decibel_20/331/pasfly/XC234332.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
Before: (1, 331, 331, 1)
After: (331, 331) 331XC164253 already in training dataset 331
Before:
 (1, 331, 331, 1)/project/data/train_data_decibel_20/331/wilsni1/XC164253.1.png

After: (33

After: (331, 331)/project/data/train_data_decibel_20/331/pasfly/XC291403.1.png does not exist, and is not a directory 
331 331
/project/data/val_data_decibel_20/331/amered/XC135462.0.png does not exist, and is not a directory
/project/data/train_data_decibel_20/331/commer/XC171600.1.png does not exist, and is not a directory
XC185863 already in training dataset
/project/data/train_data_decibel_20/331/wilsni1/XC185863.png
XC185866 already in training dataset
/project/data/train_data_decibel_20/331/wilsni1/XC185866.png
XC186352 already in training dataset
/project/data/train_data_decibel_20/331/wilsni1/XC186352.png
XC187230 already in training dataset
/project/data/train_data_decibel_20/331/wilsni1/XC187230.pngBefore: (1, 331, 330, 1)

After:XC187231 already in training dataset
 (331, 330)/project/data/train_data_decibel_20/331/wilsni1/XC187231.png 
330XC187233 already in training dataset
 331/project/data/train_data_decibel_20/331/wilsni1/XC187233.png
padded to:
 (331, 331) XC187235 alr

(331, 331) XC186284 already in validation dataset331
 /project/data/val_data_decibel_20/331/blugrb1/XC186284.0.png331

/project/data/val_data_decibel_20/331/amered/XC135463.1.png does not exist, and is not a directory
/project/data/val_data_decibel_20/331/blugrb1/XC186284.0.png does not exist, and is not a directory
XC233263 already in training dataset
/project/data/train_data_decibel_20/331/wilsni1/XC233263.1.png
/project/data/train_data_decibel_20/331/wilsni1/XC233263.1.png does not exist, and is not a directory
XC295810 already in training dataset
/project/data/train_data_decibel_20/331/vigswa/XC295810.0.png
/project/data/train_data_decibel_20/331/vigswa/XC295810.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
Before: (1, 331, 331, 1)
XC297477 already in training dataset
After: /project/data/train_data_decibel_20/331/pasfly/XC297477.1.png(331, 331) 331
 331
/project/data/train_data_decibel_20/331/pasfly/XC297477.1.png does not exist, a

...

/project/data/train_data_decibel_20/331/whbnut/A.XC446576.ts0.5.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC467716 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC467716.gn1.1.png
/project/data/train_data_decibel_20/331/olsfly/A.XC467716.gn1.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC446576 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC446576.ts0.5.1.png
/project/data/train_data_decibel_20/331/whbnut/A.XC446576.ts0.5.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC467716 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC467716.ts0.5.0.png
/project/data/train_data_decibel_20/331/olsfly/A.XC467716.ts0.5.0.png does not exist, and is not a directory
XC446754 already in training dataset
/proje

/project/data/train_data_decibel_20/331/whbnut/A.XC451418.gn1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC478455 already in validation dataset
/project/data/val_data_decibel_20/331/olsfly/A.XC478455.ex20.png
/project/data/val_data_decibel_20/331/olsfly/A.XC478455.ex20.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC451418 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC451418.ts0.5.png
/project/data/train_data_decibel_20/331/whbnut/A.XC451418.ts0.5.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC478455 already in validation dataset
/project/data/val_data_decibel_20/331/olsfly/A.XC478455.fs2.png
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
/project/data/val_data_decibel_20/331/olsfly/A.XC478455.fs2.png does not exist, and is not a directory
XC451422 already in training dataset
/project/data/train_data_dec

Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC468235 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC468235.ts0.5.png
/project/data/train_data_decibel_20/331/whbnut/A.XC468235.ts0.5.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC478823 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC478823.ts0.5.1.png
/project/data/train_data_decibel_20/331/olsfly/A.XC478823.ts0.5.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC474852 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC474852.ex20.0.png
/project/data/train_data_decibel_20/331/whbnut/A.XC474852.ex20.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC482033 already in validation dataset
/project/data/val_data_decibel_20/331/olsfly/A.XC482033.ex20.png
/project/data/val_data_decibel_20/331/olsfly/A.XC4

After: (331, 331) 331/project/data/train_data_decibel_20/331/olsfly/A.XC484154.ts0.5.0.png does not exist, and is not a directory 
331
XC490201 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC490201.ts0.5.0.png
/project/data/train_data_decibel_20/331/whbnut/A.XC490201.ts0.5.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC484154 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC484154.ts0.5.1.png
/project/data/train_data_decibel_20/331/olsfly/A.XC484154.ts0.5.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC490201 already in training dataset
/project/data/train_data_decibel_20/331/whbnut/A.XC490201.ts0.5.1.png
/project/data/train_data_decibel_20/331/whbnut/A.XC490201.ts0.5.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC487482 already in training dataset
/project/data/train_data_de

/project/data/train_data_decibel_20/331/olsfly/A.XC489325.fs2.1.png
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.fs2.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC489325 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.gn1.0.png
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.gn1.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC489325 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.gn1.1.png
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.gn1.1.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: (331, 331) 331 331
XC489325 already in training dataset
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.ts0.5.0.png
/project/data/train_data_decibel_20/331/olsfly/A.XC489325.ts0.5.0.png does not exist, and is not a directory
Before: (1, 331, 331, 1)
After: 